<a href="https://colab.research.google.com/github/rishitap25/ML-Bootcamp/blob/main/pneumonia_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🔍 Objective:
This project will **introduce you to CNNs**.

### 📌 What You’ll Do:
1. Define suitable transforms/augmentations for your `train` and `test` images.
2. Pass these images into PyTorch `DataLoaders` for batch processing.
3. Implement `CNN` class architecture for pneumonia image classification.
4. Train and validate your model.

💡 **PLEASE PLEASE PLEASE look things up!!! This is YOUR learning experience.**

---

In [15]:
import torch
import kagglehub
import torch.nn as nn

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

train_path = f'{path}/chest_xray/train/'
test_path = f'{path}/chest_xray/test/'

#### 📌 ***TASK 1 - DATA PREPROCESSING***

Define image augmentations in the cell below using two variables:  

- **`transform_train`**: Stores transforms for training images. You can include any augmentations you prefer.  
- **`transform_test`**: Stores transforms for your test images. As a best practice, limit these transformations to only the essential ones from `transform_train`.

Lastly, be sure to convert all images to [tensors](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) via the `transforms.ToTensor()` transform. Don't know transforms? [Click here](https://pytorch.org/vision/stable/transforms.html).

In [17]:
from torchvision import transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Now, we'll pass our images and transforms into a `DataLoader`, which allows us to train our model in batches.
Most of the code is done for you, but [click here](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-and-_EL_nBO9TS694cbTEl5M.A) to learn more.

In [18]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

num_batches = 32  # Feel free to change this

train_dataset = ImageFolder(root=train_path, transform=transform_train) # Your Transformed train images
train_loader = DataLoader(dataset=train_dataset, batch_size=num_batches, shuffle=True)

test_dataset = ImageFolder(root=test_path, transform=transform_test) # Your transformed test images
test_loader = DataLoader(dataset=test_dataset, batch_size=num_batches, shuffle=True)

#### 📌 ***TASK 2 - CNN Architecture***

This is where you have all the creative freedom in the world. Here are some good questions to ask yourself:

- How many [channels](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-49AG77e4Qp2e7EkARdFsTA) should go into the input layer?
- What measures can I take to avoid [overfitting](https://www.perplexity.ai/search/i-m-a-student-at-naiss-mlb-wha-YdAbhqQzRZaEq39BEQzA6w)?
- What matters to me? (Training Speed / Performance tradeoffs)
- **CONVOLUTION. ACTIVATION FUNCTION. POOLING!!!** 📢📢📢

Not comfortable with PyTorch? [Here](https://youtu.be/mozBidd58VQ?si=TE2_81TEQko1eDXT). Go and make me the best [CNN](https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns) I've ever seen :)

In [19]:
import torch
import torch.nn

# Remember what Pooling does to feature maps!!!
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()

        # Define CNN layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 3 input channels (RGB), 32 output channels
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 1)  # Output 1 for binary classification (Normal or Pneumonia)

        # Activation function
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Forward pass through the network
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        # Flatten the output for the fully connected layers
        x = x.view(-1, 128 * 28 * 28)

        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))  # Sigmoid for binary classification

        return x
...

Ellipsis

#### 📌 ***TASK 3 - DEFINE TRAIN FUNCTION***  

Define `process_forward_phase` and `train` to update model weights with each new [epoch/iteration](https://www.perplexity.ai/search/i-m-a-curious-naiss-mlb-studen-7SJECNYrS1iYxUR032dp7A). Here are the steps:

- **Forward pass:** Here, our batch is taken through the network to output a prediction (Normal/Pneumonia)
- **Backward pass:** The model goes "What's our loss? Hmmm... Not quite what I want. This means my `weights` aren't adjusted properly. Let me propagate my `loss` backward in hopes of correcting my weights."

We use **`f1_score`** as the primary metric and also display **`accuracy`** for comparison. Most steps are outlined for you—just follow the structure provided!

In [20]:
from tqdm import tqdm # Visualize training progress
from sklearn.metrics import f1_score, accuracy_score

def process_forward_phase(model, batch, criterion):
    """
    This is a helper function to abstract the "forward"
    phase of the training loop. This function also returns
    the loss, predictions, and labels seen in the batch.
    """

    images, labels = batch
    labels = labels.float()

    outputs = model(images)

    # Calculate loss
    loss = criterion(outputs.squeeze(), labels)

    # Get predictions
    preds = (outputs.squeeze() > 0.5).float()
    ...

    return loss, preds, labels

def train(model, train_loader, criterion, optimizer, epochs):
    ... # Set model to training mode

    for epoch in range(epochs): # Specify the number of iterations to train for
        all_preds, all_labels = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            optimizer.zero_grad()
            loss, preds, labels = process_forward_phase(model, batch, criterion)

            ... # (backward phase, upd. weights)

            loss.backward()
            optimizer.step()

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds)

        print(f"Epoch {epoch+1} - Acc={accuracy:.2f}%, F1={f1:.4f}")

After your model trains, you want to see how well it performs on **unseen data.** Meaning, if this were a live hospital NEEDING your predictions to classify patients with pneumonia, how well would it do?


You simply have to run this cell; all the code is implemented for you (Assuming `process_forward_phase` works fine). 😊

In [21]:
def test(model, test_loader, criterion):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            loss, preds, labels = process_forward_phase(model, batch, criterion)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Final Test Results: Acc={accuracy:.2f}%, F1={f1:.4f}")

#### 📌***TASK 4 - TRAIN MODEL***

We're close!!! We simply need to instantiate the `model`, define a suitable `criterion` (loss), and use an `optimizer` (thing to speed up backpropagation).

In [22]:
import torch.optim as optim

model = PneumoniaCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # Weight decay is optional. But is it? 🤔

train(model, train_loader, criterion, optimizer, epochs=10)

Epoch 1/10: 100%|██████████| 163/163 [19:37<00:00,  7.23s/it]


Epoch 1 - Acc=0.74%, F1=0.8500


Epoch 2/10: 100%|██████████| 163/163 [19:28<00:00,  7.17s/it]


Epoch 2 - Acc=0.74%, F1=0.8525


Epoch 3/10: 100%|██████████| 163/163 [19:13<00:00,  7.08s/it]


...

Epoch 8 - Acc=0.95%, F1=0.9687


Epoch 9/10: 100%|██████████| 163/163 [20:34<00:00,  7.57s/it]


Epoch 9 - Acc=0.95%, F1=0.9686


Epoch 10/10: 100%|██████████| 163/163 [22:27<00:00,  8.26s/it]

Epoch 10 - Acc=0.95%, F1=0.9692


Last step: evaluate your model's performance. Remember, you get **1,000,000** brownie points 🍫 if you beat Adam's **`f1_score:`0.8549**.

In [23]:
test(model, test_loader, criterion)

Final Test Results: Acc=0.75%, F1=0.8296
